In [1]:
from data.data_utils import get_distillation_dataloader
from models.models_utils import get_distillation_models
from configs import dataset as DATA_CONFIG
from configs import fsdp_config as FSDP_CONFIG
from configs import train_config as TRAIN_CONFIG
from configs import distillation_config as DISTIL_CONFIG

/home/brimmann/works/llm-recipes/models/checkpoint_handler.py:7: DeprecationWarning: `torch.distributed._shard.checkpoint` will be deprecated, use `torch.distributed.checkpoint` instead
  import torch.distributed._shard.checkpoint as dist_cp
/home/brimmann/works/llm-recipes/models/tools.py:5: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import packaging


In [2]:
train_config, fsdp_config, distil_config, data_config = TRAIN_CONFIG(), FSDP_CONFIG(), DISTIL_CONFIG(), DATA_CONFIG()

In [3]:
args_dict = {'model_name': 'EleutherAI/pythia-410m-deduped', 'dataset.file': 'datasets-m/loader/squad.py', 'lr': 1e-06, 'num_epochs': 5, 'batch_size_training': 1, 'val_batch_size': 1, 'output_dir': 'train/output/path', 'distillation_config.model_name': 'mistralai/Mistral-7B-Instruct-v0.2', 'distillation': True, 'distillation_config.enable_fsdp': False, 'distillation_config.pure_bf16': True, 'distillation_config.distil_factor': 1.5, 'save_step': 100, }

In [4]:
def update_config(config, update_dict, isSubmodule=False):
    if isinstance(config, (tuple, list)):
        for c in config:
            update_config(c, update_dict, isSubmodule)
    else:
        for k, v in update_dict.items():
            if "." in k:
                config_name, param_name = k.split(".", 1)
                if type(config).__name__ == config_name:
                    if hasattr(config, param_name):
                        setattr(config, param_name, v)
            elif not isSubmodule and hasattr(config, k):
                setattr(config, k, v)


In [5]:
update_config((train_config, fsdp_config, data_config), args_dict)
update_config((distil_config), args_dict, isSubmodule=True,)

In [6]:
student_tokenizer, teacher_tokenizer, model = get_distillation_models(train_config, distil_config, fsdp_config, 0, args_dict)

The `GPTNeoXSdpaAttention` class is deprecated in favor of simply modifying the `config._attn_implementation`attribute of the `GPTNeoXAttention` class! It will be removed in v4.48


--> Model EleutherAI/pythia-410m-deduped

--> EleutherAI/pythia-410m-deduped has 405.334016 Million params



Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

: 

In [ ]:
train_dataloader, teacher_train_dataloader, eval_dataloader, teacher_eval_dataloader = get_distillation_dataloader(data_config, train_config, distil_config, student_tokenizer, teacher_tokenizer, 0)

In [1]:
from datasets import load_from_disk
d = load_from_disk("mistral-sqaud-ds")

In [2]:
d

Dataset({
    features: ['context', 'question', 'answers', 'answers_generated'],
    num_rows: 5
})

In [4]:
for it in d:
    print(it)

{'context': 'Time has long been a major subject of study in religion, philosophy, and science, but defining it in a manner applicable to all fields without circularity has consistently eluded scholars. Nevertheless, diverse fields such as business, industry, sports, the sciences, and the performing arts all incorporate some notion of time into their respective measuring systems. Some simple definitions of time include "time is what clocks measure", which is a problematically vague and self-referential definition that utilizes the device used to measure the subject as the definition of the subject, and "time is what keeps everything from happening at once", which is without substantive meaning in the absence of the definition of simultaneity in the context of the limitations of human sensation, observation of events, and the perception of such events.', 'question': 'Time has long been a major point of study in which fields?', 'answers': {'answer_start': [47], 'text': ['religion, philoso